In [ ]:
# imports and helpers
import re, base64, requests, io
from IPython.display import display, HTML, Javascript, clear_output
import ipywidgets as widgets

def normalize_to_data_url(text, mime):
    t = text.strip()
    if not t:
        return None
    if t.startswith('data:'):
        return t
    if re.match(r'^https?://', t, re.I):
        return t
    compact = re.sub(r'\s+', '', t)
    # len%4 not strict; try decode
    try:
        # validate base64 by attempting decode
        base64.b64decode(compact, validate=True)
        return f'data:{mime};base64,{compact}'
    except Exception:
        # try forgiving decode (no validate)
        try:
            base64.b64decode(compact)
            return f'data:{mime};base64,{compact}'
        except Exception:
            return None


In [ ]:
# UI widgets
textarea = widgets.Textarea(placeholder='Paste a data URL or raw base64 here', layout=widgets.Layout(width='100%', height='220px'))
mime_dropdown = widgets.Dropdown(options=['image/png','image/jpeg','image/webp','image/svg+xml','image/gif','application/octet-stream'], value='image/png', description='MIME:')
filename_input = widgets.Text(placeholder='filename (e.g. picture.png)', layout=widgets.Layout(width='50%'))
render_btn = widgets.Button(description='Render')
clear_btn = widgets.Button(description='Clear', button_style='')
download_btn = widgets.Button(description='Download', button_style='')
open_btn = widgets.Button(description='Open in new tab', button_style='')
copy_btn = widgets.Button(description='Copy data URL')
output = widgets.Output(layout=widgets.Layout(border='1px solid #eee', padding='8px'))

display(widgets.VBox([textarea, widgets.HBox([mime_dropdown, filename_input]), widgets.HBox([render_btn, clear_btn, download_btn, open_btn, copy_btn]), output]))


In [ ]:
# Handlers
current_data_url = {'value': None}

def show_meta_and_image(data_url, inferred_type=None, inferred_size=None):
    with output:
        clear_output()
        if data_url is None:
            display(HTML('<div style="color:#666;font-size:13px">No image rendered yet.</div>'))
            return
        # build meta
        if data_url.startswith('data:'):
            m = re.match(r'^data:([^;]+);base64,', data_url)
            typ = m.group(1) if m else (inferred_type or mime_dropdown.value)
            # estimate size
            b64 = data_url.split(',',1)[1]
            size_kb = round(len(b64)*3/4/1024)
            filename = filename_input.value.strip() or 'image'
            meta_html = f"<strong>{filename}</strong><br>{typ} — ~{size_kb} KB"
        else:
            typ = inferred_type or 'remote'
            meta_html = f"<strong>{filename_input.value.strip() or 'image'}</strong><br>{typ} — remote URL"
        img_html = f'<img src="{data_url}" style="max-width:320px;max-height:320px;border-radius:8px;border:1px solid #f0f0f6;padding:6px;display:block">'
        display(HTML(f'<div style="display:flex;gap:12px;align-items:flex-start">{img_html}<div style="font-size:13px;color:#666;margin-left:8px">{meta_html}</div></div>'))

def render_action(b):
    raw = textarea.value
    mime = mime_dropdown.value
    du = normalize_to_data_url(raw, mime)
    if not du:
        with output:
            clear_output()
            display(HTML('<div style="color:red">Could not detect valid base64 or data URL.</div>'))
        current_data_url['value'] = None
        return
    # if remote URL, fetch and convert to data URL
    if re.match(r'^https?://', du, re.I) and not du.startswith('data:'):
        try:
            r = requests.get(du, timeout=10)
            r.raise_for_status()
            content_type = r.headers.get('Content-Type','')
            if ';' in content_type:
                content_type = content_type.split(';',1)[0]
            b64 = base64.b64encode(r.content).decode('ascii')
            data_url = f'data:{content_type or mime};base64,{b64}'
            current_data_url['value'] = data_url
            show_meta_and_image(data_url, inferred_type=content_type, inferred_size=len(r.content))
        except Exception as e:
            with output:
                clear_output()
                display(HTML(f'<div style="color:red">Could not fetch image: {str(e)}</div>'))
            current_data_url['value'] = None
    else:
        current_data_url['value'] = du
        show_meta_and_image(du)

def clear_action(b):
    textarea.value = ''
    filename_input.value = ''
    current_data_url['value'] = None
    show_meta_and_image(None)

def download_action(b):
    du = current_data_url['value']
    if not du:
        with output:
            clear_output()
            display(HTML('<div style="color:red">Nothing to download.</div>'))
        return
    # create anchor that triggers download via JS
    fname = filename_input.value.strip() or 'image'
    if du.startswith('data:'):
        mime_match = re.match(r'^data:([^;]+);base64,', du)
        ext = 'png'
        if mime_match:
            parts = mime_match.group(1).split('/')
            if len(parts)>1:
                ext = parts[1].replace('+','-')
        if '.' not in fname:
            fname = f'{fname}.{ext}'
        js = """var a = document.createElement('a');
a.href = '%s';
a.download = '%s';
document.body.appendChild(a);
a.click();
a.remove();""" % (du, fname)
        display(Javascript(js))
    else:
        # remote url: open in new tab (user can save)
        display(Javascript("window.open('%s', '_blank')" % du))

def open_action(b):
    du = current_data_url['value']
    if not du:
        with output:
            clear_output()
            display(HTML('<div style="color:red">Nothing to open.</div>'))
        return
    display(Javascript("window.open('%s', '_blank')" % du))

def copy_action(b):
    du = current_data_url['value']
    if not du:
        with output:
            clear_output()
            display(HTML('<div style="color:red">Nothing to copy.</div>'))
        return
    js = "navigator.clipboard.writeText('%s').then(()=>{console.log('copied')}).catch(e=>alert('Copy failed: '+e))" % du.replace("'", "\'")
    display(Javascript(js))

render_btn.on_click(render_action)
clear_btn.on_click(clear_action)
download_btn.on_click(download_action)
open_btn.on_click(open_action)
copy_btn.on_click(copy_action)

# initial
show_meta_and_image(None)
